In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Box, Layout, Output, Label
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from IPython.display import display, Latex, Markdown, Image, Math

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

## Regelung von Totzeitsystemen

In [ ]:
imag = Image("../images/smith.png", width=800)

outL = Output()
outR = Output()

with outL:
    display(imag)
with outR:
    display(Markdown("""
- lineares zeitverzögertes System im Zeitbereich
  \\begin{align*}
      \\dot{x}(t) & = -\\frac{1}{T} x(t) + \\frac{K_\\text{S}}{T} u(t - \\tau)
  \\end{align*}
- Darstellung im Bildbereich
  \\begin{align*}
      G(s) & = \\frac{K_\\text{S}}{1 + T s} e^{-s \\tau}
  \\end{align*}
- Regelung mittels 
    - PI-Regler
    - Smith-Prädiktor mit PI
    - Prädiktor-Regelung
    """))
cols = HBox([outL, outR], layout=Layout(display='flex', flex_flow='row', justify_content='space-around', align_items='center'))
display(cols)

__Definition Parameter__

In [ ]:
tSim = np.linspace(0, 500, 501)

In [ ]:
T = 68.21
Tt = 50.0
KS = 0.15

## Simulation offener Kreis

In [ ]:
x0 = [0]
u = lambda t: 1 if t >= 0 else 0

In [ ]:
def process(t, x, u, KS, T):
    dx = -1/ T * x[0] + KS / T * u(t)
    return dx

In [ ]:
def processDelay(t, x, u, KS, T, Tt):
    dx = -1/ T * x[0] + KS / T * u(t - Tt)
    return dx

In [ ]:
res = solve_ivp(process,
                [tSim[0], tSim[-1]],
                x0,
                t_eval=tSim, args=(u, KS, T))

In [ ]:
resDelay = solve_ivp(processDelay,
                     [tSim[0], tSim[-1]],
                     x0,
                     t_eval=tSim, args=(u, KS, T, Tt))

In [ ]:
plt.close()

fig, axes10 = plt.subplots(1, 1, figsize=(10,6))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

axes10.plot(res.t, res.y[0, :], 'C0', label=r"unverzögerte Strecke")
axes10.plot(resDelay.t, resDelay.y[0, :], 'C0--', label=r"verzögert Strecke")

axes11 = axes10.twinx()
axes11.plot(tSim, [u(t) for t in tSim], 'C4')
axes11.plot(tSim, [u(t - Tt) for t in tSim], 'C4--')
axes11.set_ylabel(r"$\tilde{u}_{\mathrm{A}}$ in V", color='C4')
axes11.tick_params(axis='y', colors='C4')

axes10.set_xlabel(r"$t$ in s")
axes10.set_ylabel(r"$y$")

axes10.grid() 

handlesAx, labelsAx = axes10.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.90, 0.775, .15), loc=3,
           ncol=4, mode="expand", borderaxespad=0., framealpha=0.5)
plt.show()

## Simulation geschlossener Kreis

In [ ]:
tSim = np.linspace(0, 1000, 1001)

In [ ]:
def processDelayCtrl(t, x, KS, T, Tt, tSim, uAll):
    u = np.interp(t - Tt, tSim, uAll)

    dx = -1/ T * x[0] + KS / T * u
    
    return dx

In [ ]:
def processDelaySmith(t, x, KS, T, Tt, Ki, tSim, uAll):
    u = np.interp(t - Tt, tSim, uAll)
    uDF = np.interp(t, tSim, uAll)
    
    dx = np.zeros(2)
    dx[0] = -1/ T * x[0] + KS / T * u
    dx[1] = -1/ T * x[1] + KS / T * uDF
    
    return dx

In [ ]:
output = widgets.Output()

with output:
    fig, ax1 = plt.subplots(1, 1, figsize=(10, 6))

fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([0, tSim[-1]])
ax1.set_ylim([-0.01, 0.40]) 
ax1.grid()
ax1.set_ylabel(r"$y$")
ax1.set_xlabel(r"$t$ in s")
ax11 = ax1.twinx()
ax11.set_ylim([-0.25, 0.25]) 
ax11.set_ylabel(r'$u$', color='C1')
ax11.tick_params(axis='y', colors='C1')

yRefSys, = ax1.plot([], [], 'C0', label=r'Referenz')
yRefDelSys, = ax1.plot([], [], 'C0:', alpha=0.5)
ySys, = ax1.plot([], [], 'C2', label=r'System')
uSys, = ax11.plot([], [], 'C1:')

handlesAx, labelsAx = ax1.get_legend_handles_labels()

fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.13, 0.94, 0.7675, .15), loc=3,
           ncol=6, mode="expand", borderaxespad=0., framealpha=0.5)

sliderT0 = widgets.widgets.FloatSlider(value=0.,
                                       min=0.,
                                       max=500,
                                       step=1,
                                       description=r"$t_0$")
sliderYd = widgets.widgets.FloatSlider(value=0.15,
                                       min=0.05,
                                       max=0.5,
                                       step=0.05,
                                       description=r"$y_\text{d}$")
sliderT = widgets.widgets.FloatSlider(value=1,
                                       min=1,
                                       max=50,
                                       step=1,
                                       description=r"$\Delta t$")
sliderKp = widgets.FloatSlider(value=14,
                               min=1,
                               max=20,
                               step=0.1,
                               continuous_update=True,
                               description=r'$K_{\mathrm{P}}$')
sliderKi = widgets.FloatSlider(value=0.45,
                               min=0.01,
                               max=1,
                               step=0.001,
                               continuous_update=True,
                               description=r'$K_{\mathrm{I}}$')

radioU = widgets.RadioButtons(options=['PI-Regler', 'PI-Regler Z/N', 'Smith-Prädiktor mit PI-Regler', 'Prädiktor-Steuerung', 'Prädiktor-Regelung'],
                              disabled=False)

def calcDelay(yRef, Kp, Ki):
    yDelayCtrl = np.zeros(len(tSim))
    uDelayCtrl = np.zeros(len(tSim))
    eSum = 0
    dt = tSim[1] - tSim[0]
    x0 = [0]

    for idx, x in enumerate(tSim[:-1]):
        resDelayCtrl = solve_ivp(processDelayCtrl,
                                 [tSim[idx], tSim[idx+1]],
                                 x0,
                                 args=(KS, T, Tt, tSim[:idx+1], uDelayCtrl[:idx+1]))
        x0 = resDelayCtrl.y.T[-1, :]
        y = resDelayCtrl.y.T[-1, 0]

        e = yRef(x) - y
        eSum += e * dt
        u = Kp * e + Ki * eSum

        yDelayCtrl[idx + 1] = y
        uDelayCtrl[idx + 1] = u
    
    return tSim, yDelayCtrl, uDelayCtrl

def calcSmith(yRef, Kp, Ki):
    yDelaySmith = np.zeros(len(tSim))
    yDelaySmithDF = np.zeros(len(tSim))
    uDelaySmith = np.zeros(len(tSim))

    eSum = 0
    dt = tSim[1] - tSim[0]
    x0 = [0, 0]

    for idx, x in enumerate(tSim[:-1]):
        resDelaySmith = solve_ivp(processDelaySmith,
                                  [tSim[idx], tSim[idx+1]],
                                  x0,
                                  args=(KS, T, Tt, Ki, tSim[:idx+1], uDelaySmith[:idx+1]))
        x0 = resDelaySmith.y.T[-1, :]
        y = resDelaySmith.y.T[-1, 0]
        yDF = resDelaySmith.y.T[-1, 1]
        yDelaySmith[idx + 1] = y
        yDelaySmithDF[idx + 1] = yDF

        eDF = y - np.interp(tSim[idx+1] - Tt, tSim[:idx+2], yDelaySmithDF[:idx+2])
        e = yRef(x) - (yDF + eDF)
        eSum += e * dt
        u = Kp * e + Ki * eSum

        yDelaySmith[idx + 1] = y
        uDelaySmith[idx + 1] = u

    return tSim, yDelaySmith, uDelaySmith

def calcPredFf(uRef):
    dt = tSim[1] - tSim[0]
    x0 = [0]

    resDelayFf = solve_ivp(processDelay,
                           [tSim[0], tSim[-1]],
                           x0,
                           t_eval=tSim,
                           args=(uRef, KS, T, Tt))

    return tSim, resDelayFf.y.T[:, -1], np.array([uRef(t) for t in tSim])


def calcPredCtrl(yRef, dyRef, uRef, Kp, Ki):
    yDelayCtrl = np.zeros(len(tSim))
    uDelayCtrl = np.zeros(len(tSim))

    eSum = 0
    dt = tSim[1] - tSim[0]
    x0 = [0]
    N = int(Tt / dt)

    for idx, x in enumerate(tSim[:-1]):
        resDelayCtrl = solve_ivp(processDelayCtrl,
                                 [tSim[idx], tSim[idx+1]],
                                  x0,
                                  args=(KS, T, Tt, tSim[:idx+1], uDelayCtrl[:idx+1]))
        x0 = resDelayCtrl.y.T[-1, :]
        y = resDelayCtrl.y.T[-1, 0]
        resDelayCtrl[idx + 1] = y

        yPred = np.exp(-Tt / T) * y + KS / T * dt * np.sum([np.exp(-(Tt - i * dt) / T) * uDelayCtrl[idx - N + i] for i in range(N)])
        
        e = yPred - yRef(x + Tt)
        eSum += e * dt
        u = -Kp * e - Ki * eSum

        yDelayCtrl[idx + 1] = y
        uDelayCtrl[idx + 1] = u

    return tSim, yDelayCtrl, uDelayCtrl

def calcSystem(_):
    Kp = sliderKp.value
    Ki = sliderKi.value
    yd = sliderYd.value
    t0 = sliderT0.value
    dt = sliderT.value
    uType = radioU.value

    if uType == 'Prädiktor-Steuerung':
        yr = lambda t: yd * (3 * (t - (t0 - Tt) ) ** 2 / dt ** 2 - 2 * (t - (t0 - Tt)) ** 3 / dt ** 3) if t0 - Tt <= t <= t0 - Tt + dt else 0 if t < t0 - Tt else yd 
        dyr = lambda t: yd * (6 * (t - (t0 - Tt)) / dt ** 2 - 6 * (t - (t0 - Tt)) ** 2 / dt ** 3) if t0 - Tt < t < t0 - Tt + dt else 0        
        uTraj = lambda t: T / KS * (dyr(t) + yr(t) / T)
    else:
        yr = lambda t: yd * (3 * (t - t0 ) ** 2 / dt ** 2 - 2 * (t - t0) ** 3 / dt ** 3) if t0 <= t <= t0 + dt else 0 if t < t0 else yd 
        dyr = lambda t: yd * (6 * (t - t0) / dt ** 2 - 6 * (t - t0) ** 2 / dt ** 3) if t0 < t < t0 + dt else 0        
        uTraj = lambda t: T / KS * (dyr(t) + yr(t) / T)
    
    if uType == 'Smith-Prädiktor mit PI-Regler':
        tSysD, resYSysD, resUSysD = calcSmith(yr, Kp, Ki)
    elif uType == 'Prädiktor-Steuerung': 
        tSysD, resYSysD, resUSysD = calcPredFf(uTraj)
    elif uType == 'Prädiktor-Regelung':
        tSysD, resYSysD, resUSysD = calcPredCtrl(yr, dyr, uTraj, Kp, Ki)
    elif uType == 'PI-Regler Z/N':
        Kp = 0.9 * T / (KS * Tt)
        Tn = 3.3 * Tt
        Ki = Kp / Tn
        sliderKp.value = str(Kp)
        sliderKi.value = str(Ki)
        tSysD, resYSysD, resUSysD = calcDelay(yr, Kp, Ki)
    else:
        tSysD, resYSysD, resUSysD = calcDelay(yr, Kp, Ki)

    if uType == 'Prädiktor-Steuerung':
        yRefSys.set_data(tSim, [yr(t - Tt) for t in tSim])
        yRefDelSys.set_data(tSim, [yr(t) for t in tSim])
    else:
        yRefSys.set_data(tSim, [yr(t) for t in tSim])
    ySys.set_data(tSysD, resYSysD)
    uSys.set_data(tSysD, resUSysD)

    ax1.set_ylim([np.min(resYSysD) - 0.05, (np.max(resYSysD) + 0.2)]) 
    ax11.set_ylim([np.min(resUSysD) - 1, (np.max(resUSysD) + 2)]) 

    fig.canvas.draw()

sliderKp.observe(calcSystem, names='value')
sliderKp.observe(calcSystem, names='value')
sliderKi.observe(calcSystem, names='value')
sliderYd.observe(calcSystem, names='value')
sliderT.observe(calcSystem, names='value')
sliderT0.observe(calcSystem, names='value')
radioU.observe(calcSystem, names='value')

calcSystem(_)

bFeedForward = VBox([Label(value='Steuerung'), sliderYd, sliderT0, sliderT])
bFeedForward.layout = make_box_layout()
bFeedBackType = VBox([Label(value='Regler'), radioU])
bFeedBackParams = VBox([Label(value='Reglerparameter'), sliderKp, sliderKi])
bFeedBack = HBox([bFeedBackType, bFeedBackParams])
bFeedBack.layout = make_box_layout()

controls = HBox([bFeedForward, bFeedBack])
VBox([controls, output], layout=Layout(display='flex', flex_flow='column', justify_content='center', align_items='center'))